<a href="https://colab.research.google.com/github/Pentuhov/PR-AI6/blob/main/%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B9_%D0%B1%D0%BB%D0%BE%D0%BA_%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B8_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%B8_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9_%D0%94%D0%97_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На базе баскетбольных матчей добейтесь средней абсолютной ошибки 17 и менее очков.

### Подготовка  

In [1]:
# Загрузка из google облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/basketball.csv', None, quiet=True)

# Библиотека для работы с базами
import pandas as pd
df = pd.read_csv('basketball.csv', encoding= 'cp1251', sep=';', header=0, index_col=0) # Загружаем базу
df.head()

,TOTAL,info,Ком. 1,Ком. 2,Минута,Общая минута,Секунда,fcount,ftime
0,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,0.0,1,1.0,30,81,90.0
1,"100,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,1,1.0,45,81,105.0
2,"99,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,0,81,120.0
3,"98,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,2,2.0,30,81,150.0
4,"95,5",4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,3,3.0,0,81,180.0


Извлекаем текстовые данные из колонки `info` таблицы, помещаем в переменную `data_text`. Выводим длину списка:

In [2]:
data_text = df['info'].values #

len(data_text) #

52450

Задаем максимальное кол-во слов в словаре, помещаем в переменную все символы, которые хотим вычистить из текста.

 Токенизируем текстовые данные:

In [3]:
# Импортируем токенайзер
from tensorflow.keras.preprocessing.text import Tokenizer

maxWordsCount = 5000

sim_for_del='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer (num_words=maxWordsCount,
                       filters=sim_for_del,
                       lower=True,
                       split=' ',
                       oov_token='unknown',
                       char_level=False)

tokenizer.fit_on_texts(data_text)

In [4]:
# Переводим в Эмбеддинг пространство
Sequences = tokenizer.texts_to_sequences(data_text)

# Вариант  Bag of Words
xBOW_text = tokenizer.sequences_to_matrix(Sequences)

Преобразуем данные в numpy, подготовим наборы для обучения:

In [5]:
# Библиотека работы с массивами
import numpy as np

xTrain = np.array(df[['Ком. 1','Ком. 2', 'Минута', 'Секунда','ftime']].astype('int'))
yTrain = np.array(df['fcount'].astype('int'))

In [6]:
print(xTrain.shape)
print(yTrain.shape)
print(xBOW_text.shape)

(52450, 5)
(52450,)
(52450, 5000)


In [7]:
# Функция по проверке ошибки

def check_MAE_predictl_DubbleInput (model,
                                    x_data,
                                    x_data_text,
                                    y_data_not_scaled,
                                    plot=False):

  mae = 0 # Инициализируем начальное значение ошибки
  y_pred = (model.predict([x_data,x_data_text])).squeeze()

  for n in range (0,len(x_data)):
    mae += abs(y_data_not_scaled[n] - y_pred[n]) # Увеличиваем значение ошибки для текущего элемента
  mae /= len(x_data) # Считаем среднее значение
  print('Среднаяя абслолютная ошибка {:.3f} очков это {:.3f}% от общей выборки в {} игры'.format(mae, (mae/y_data_not_scaled.mean(axis=0))*100,len(x_data)))

  if plot:
     plt.scatter(y_data_not_scaled, y_pred)
     plt.xlabel('Правильные значение')
     plt.ylabel('Предсказания')
     plt.axis('equal')
     plt.xlim(plt.xlim())
     plt.ylim(plt.ylim())
     plt.plot([0, 250], [0, 250])
     plt.show()

In [8]:
def on_epoch_end_custom(epoch, logs=None):
    check_MAE_predictl_DubbleInput(model_final_scaled,xTrain_scaled,xBOW_text,yTrain,plot=True)

In [ ]:
# ваше решение

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Concatenate, Flatten

# Определение входов
input_numeric = Input(shape=(xTrain.shape[1],))
input_text = Input(shape=(xBOW_text.shape[1],))

# Вход для текста
x_text = Dense(128, activation='relu')(input_text)
x_text = Flatten()(x_text)

# Соединяем оба ввода
combined = Concatenate()([input_numeric, x_text])

# Полносвязные слои
z = Dense(64, activation='relu')(combined)
z = Dense(32, activation='relu')(z)
output = Dense(1)(z)  # Один выход для предсказания очков

# Создаем модель
model_final_scaled = Model(inputs=[input_numeric, input_text], outputs=output)
model_final_scaled.compile(optimizer='adam', loss='mean_absolute_error')

# Обучение модели
model_final_scaled.fit([xTrain, xBOW_text], yTrain, epochs=50, batch_size=32)

Epoch 1/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 24.7497
Epoch 2/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 2.5339
Epoch 3/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 2.4212
Epoch 4/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2.1431
Epoch 5/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2.0136
Epoch 6/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 1.8272
Epoch 7/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.7742
Epoch 8/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.5199
Epoch 9/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.5456
Epoch 10/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.4637
Epoch 11/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.4029
Epoch 12/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 1.3467
Epoch 13/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.2606
Epoch 14/50
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.2668
Epoch 15/50
1640/1640 ━━━━━

In [10]:
check_MAE_predictl_DubbleInput(model_final_scaled, xTrain, xBOW_text, yTrain, plot=True)

1640/1640 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Среднаяя абслолютная ошибка 0.622 очков это 0.411% от общей выборки в 52450 игры


NameError: name 'plt' is not defined

Для решения задачи требуются знания по теме `Callbacks`, пожалуйста, изучите раздел:

---
Дополнителная информация: ([База знаний УИИ – **«Callbacks»**](https://colab.research.google.com/drive/1COE1R3kFhRlxPm9X6hQMvjKoiFFt83xW?usp=sharing))

---